In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import json
import sys
import pandas as pd
import re

In [2]:
from dataclasses import dataclass

@dataclass
class YoutubeLink:
    url: str
    subreddit: str
    thread_url: str

In [3]:
SUBREDDITS = ['investing', 'finance', 'cryptocurrency', 'wallstreetbets']
YoutubeLinks = []

driver = webdriver.Firefox()

In [4]:
def process_thread(url):
    # old.reddit.com is easier to scrape, doesn't have js lazy loading
    driver.get(url)
    soup = BeautifulSoup(driver.page_source)
    
    #Find all links in comments, keeping only youtube links
    comments = soup.find_all('div', {"data-type": "comment"})
    links = []
    for comment in comments:
        for link in comment.find('div', {'class': 'usertext-body'}).find_all('a', href=True):
            if 'youtube.com' in link['href']:
                links.append(link['href'])
    
    return links


In [5]:
for SUBREDDIT in SUBREDDITS:
    url = f"https://reddit.com/r/{SUBREDDIT}/top/?sort=top&t=year"
    driver.get(url)
    
    # Get all thread links
    soup = BeautifulSoup(driver.page_source)
    links = soup.find_all('a', {"data-click-id": "body"})
    threads = [link for link in links if link['href'].startswith('/r/')]
    
    for thread in threads:
        thread_url = f"https://old.reddit.com{thread['href']}"
        links = process_thread(thread_url)
        
        if len(links) > 0:
            for link in links:
                YoutubeLinks.append(YoutubeLink(link, SUBREDDIT, thread_url))
                
driver.close()

In [6]:
pd.DataFrame(YoutubeLinks).to_csv('../data/raw/scraped_youtube_links.csv', index=False)